In [2]:
from IPython.display import Latex, display, Math

import sympy as sy
from sympy.interactive import printing;
from sympy.abc import x, k;
from sympy import Float;

from scipy.integrate import quad

from sympy.utilities.lambdify import lambdastr;

import numpy as np

import math

import ipywidgets as widgets
from ipywidgets import interact, widgets, Layout
from ipywidgets import interactive, interact_manual, interact;

import matplotlib.pyplot as mpl

from IPython.display import Latex

printing.init_printing(use_latex=True);
mpl.rcParams['figure.figsize'] = 7, 7



In [49]:
def geta0(f,a,b,l):
    a0 = 0
    for i in range(len(f)):
        a0 += quad(sy.lambdify(x,f[i]),a[i],b[i])[0]
    return (1/(2*l))*a0

def getcoeff(f,a,b,l,n):
    ak = []
    bk = []
    for i in range(len(f)):
        fa = f[i]*(sy.cos((k*x*sy.pi)/l))
        fb = f[i]*(sy.sin((k*x*sy.pi)/l))
        
        ffa = eval(lambdastr((x,k),fa))
        ffb = eval(lambdastr((x,k),fb))
        
        for j in range(n):
            if(i==0):
                ak.append((1/l)*quad(ffa,a[i],b[i],args=(j+1), epsabs = 0.1)[0])
                bk.append((1/l)*quad(ffb,a[i],b[i],args=(j+1), epsabs = 0.1)[0])
            else:
                ak[j]+=(1/l)*quad(ffa,a[i],b[i],args=(j+1))[0]
                bk[j]+=(1/l)*quad(ffb,a[i],b[i],args=(j+1))[0]     
    return (ak, bk)


def roundToZero(m,e):
    for i in range(len(m)):
            if (abs(m[i]) < e):
                m[i] = 0
                
                
#Función invocada por el widget
def displayOutput(N):
    fig, ax = mpl.subplots(figsize=(10, 10));
    coeffs = getcoeff(f,a,b,l,N)
    
    roundToZero(coeffs[0],0.00000000001)
    roundToZero(coeffs[1],0.00000000001)
        
    fa = geta0(f,a,b,l)

    #Graficando las funciones reales
    for i in range(m):
        xv = np.linspace(a[i], b[i], 110)
        if isinstance(f[i].evalf(), sy.numbers.Float):
            lambdaf = np.vectorize(sy.lambdify(x,f[i]))
        else:
            lambdaf = sy.lambdify(x,f[i])
        ax.plot(xv, lambdaf(xv), color='green')

    #generando polinomio
    for j in range(N):
        fa += coeffs[0][j]*sy.cos(((j+1)*sy.pi*x)/l) + coeffs[1][j]*sy.sin(((j+1)*sy.pi*x)/l)

    #graficando la funcion obtenida    
    lambdaf1 = sy.lambdify(x,fa)
    xv1 = np.linspace(min(a), max(b), 110)
    ax.plot(xv1, lambdaf1(xv1), color='r')
    
    mpl.show()
    display(fa.evalf())

In [53]:
f = []
a = []
b = []

while True:
    m = int(input('Cantidad de funciones: '))
    if m > 0:
        break

for c in range(1,m+1):    
    while True:
        try:
            f.append(sy.sympify(input("Funcion "+str(c)+": ")))
            break
        except sy.SympifyError:
            print("La función no fue ingresada correctamente. Intente de nuevo.")
            
    while True:
        try:
            a.append(float(sy.sympify(input("Desde: ")).evalf()))
            b.append(float(sy.sympify(input("Hasta: ")).evalf()))
            break
        except ValueError:
            print("El valor ingresado no es un número. Intente de nuevo.")

while True:
    try:
        T = float(sy.sympify(input("Periodo :")).evalf())
        break
    except ValueError:
        print("El valor ingresado no es un número. Intente de nuevo.")
        
display(f)

Cantidad de funciones: 3
Funcion 1: x
Desde: 0
Hasta: 1
Funcion 2: 1
Desde: 1
Hasta: 2
Funcion 3: 3-x
Desde: 2
Hasta: 3
Periodo :3


In [52]:
l=T/2
interactive_plot = interact_manual(displayOutput, N=(1, 20, 1))

interactive(children=(IntSlider(value=10, description='N', max=20, min=1), Button(description='Run Interact', …